In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 7


In [2]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [3]:
loader = GenericLoader.from_filesystem(
    "./DotNetFramework/",
    glob="*",
    suffixes=[".cs"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=1000),    
)
docs = loader.load()

In [19]:
docs[0].metadata['source']

'DotNetFramework/file1.cs'

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=1000, chunk_overlap=0
)
documents = code_splitter.split_documents(docs)



In [6]:
documents[0].page_content

'using System;\nusing System.Web.UI;\nusing System.Web.UI.WebControls;\n\nnamespace WebFormsApp\n{\n    public class CustomWebControl : WebControl\n    {\n        protected override void RenderContents(HtmlTextWriter writer)\n        {\n            writer.Write("Hello, this is a custom WebControl for .NET Framework WebForms.");\n        }\n    }\n\n    public partial class WebForm1 : Page\n    {\n        protected void Page_Load(object sender, EventArgs e)\n        {\n            CustomWebControl myControl = new CustomWebControl();\n            myControl.ID = "MyCustomControl";\n            this.form1.Controls.Add(myControl);\n        }\n    }\n}'

In [7]:
## FAISS Vector Database
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [8]:
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")

from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3.5-turbo")


In [9]:
# ## Design ChatPrompt Template
# from langchain_core.prompts import ChatPromptTemplate

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 5/6+ or ASP.NET Core."),
#         ("user","old code:{oldcode}")
#     ]
# )



## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 5/6+ or ASP.NET Core. 
In addition, basedon the context provided, please help to identify.Net Framework features replaced. To make it easy to extract the new code, please wrap the new code between <newcode> and </newcode>.
<context>
{context}
</context>
Old Code: {input}"""
)


In [10]:
## Chain Introduction
## Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
"""A retriever is an interface that returns documents given an unstructured query. 
It is more general than a vector store. A retriever does not need to be able to store documents,
only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, 
but there are other types of retrievers as well.
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8f5c790980>)

In [12]:
from langchain.chains import create_retrieval_chain

retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [13]:
response=retriever_chain.invoke({"input": documents[0].page_content})

In [14]:
response['answer']

'<newcode>\nusing System;\nusing Microsoft.AspNetCore.Mvc;\nusing Microsoft.AspNetCore.Mvc.RazorPages;\n\nnamespace WebFormsApp\n{\n    public class CustomWebControl : Microsoft.AspNetCore.Mvc.ViewComponents.ViewComponent\n    {\n        public IViewComponentResult Invoke()\n        {\n            return Content("Hello, this is a custom WebControl for ASP.NET Core Razor Pages.");\n        }\n    }\n\n    public partial class WebForm1 : PageModel\n    {\n        public void OnGet()\n        {\n            CustomWebControl myControl = new CustomWebControl();\n            myControl.ViewContext = this.ViewContext;\n            this.Response.BodyWriter.Write("Hello, this is a custom WebControl for ASP.NET Core Razor Pages.");\n        }\n    }\n}\n</newcode> \n\n.NET Framework features replaced: \n- System.Web.UI namespace replaced with Microsoft.AspNetCore.Mvc\n- Inheritance from WebControl replaced with inheritance from Microsoft.AspNetCore.Mvc.ViewComponents.ViewComponent\n- Rendering co

In [26]:
import re
from IPython.display import display, Markdown

# Your input string
input_str = response['answer']
code_block=None;

code_blocks = re.findall(r"<newcode>(.*?)</newcode>", input_str, re.DOTALL)

code_blocks = [code.strip() for code in code_blocks]

for index, block in enumerate(code_blocks):
    print(f"Code Block {index + 1}:")
    print(block)
    filename = docs[0].metadata['source'].replace('DotNetFramework', 'DotNet'); 
    if(index>0):
        filename = filename.replace('.cs', '_'+index+'.cs'); 
    # Open the file in write mode ('w') and write the code block to it
    with open(filename, 'w') as file:
        file.write(block)

    if(index>0):
        print("\n-----------------\n")


# Find all <newcode></newcode> blocks
matches = list(re.finditer(r"<newcode>.*?</newcode>", input_str, re.DOTALL))

# Check if there are any matches
if matches:
    # Take the end index of the last match
    last_match_end = matches[-1].end()
    # Extract the remaining part of the response after the last </newcode>
    replaced_features = input_str[last_match_end:].strip()
else:
    replaced_features = "No replaced features section found."

# print(replaced_features)


display(Markdown(replaced_features))


Code Block 1:
using System;
using Microsoft.AspNetCore.Mvc;
using Microsoft.AspNetCore.Mvc.RazorPages;

namespace WebFormsApp
{
    public class CustomWebControl : Microsoft.AspNetCore.Mvc.ViewComponents.ViewComponent
    {
        public IViewComponentResult Invoke()
        {
            return Content("Hello, this is a custom WebControl for ASP.NET Core Razor Pages.");
        }
    }

    public partial class WebForm1 : PageModel
    {
        public void OnGet()
        {
            CustomWebControl myControl = new CustomWebControl();
            myControl.ViewContext = this.ViewContext;
            this.Response.BodyWriter.Write("Hello, this is a custom WebControl for ASP.NET Core Razor Pages.");
        }
    }
}


.NET Framework features replaced: 
- System.Web.UI namespace replaced with Microsoft.AspNetCore.Mvc
- Inheritance from WebControl replaced with inheritance from Microsoft.AspNetCore.Mvc.ViewComponents.ViewComponent
- Rendering content in a WebControl replaced with returning content in a ViewComponent
- Inheritance from Page replaced with inheritance from PageModel
- Adding controls to form controls replaced with writing content directly to the response body